<a href="https://colab.research.google.com/github/omaraziz255/MRNet/blob/master/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive, files
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import models,layers,losses,optimizers, backend
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder
from scipy import ndimage as nd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, BatchNormalization, Activation,DepthwiseConv2D, Input, Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import binary_accuracy, mae
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import VGG16
import warnings
backend.set_image_data_format('channels_last')
drive.mount('/content/gdrive/')

In [0]:
def transfer_model_train(basepath, plane, model,label):

  path = basepath + "/" + plane
  files = [f for f in sorted(os.listdir(path)) if f.endswith(".npy")]
  count = 10
  labels = np.load(basepath + "/" + label + ".npy")[:,1]
  mc = ModelCheckpoint("/content/gdrive/My Drive/Models/"+"TransferVGG"+"_"+plane+"_"+label+".h5", monitor='val_accuracy',mode='max', save_best_only=False)
  es = EarlyStopping(monitor='val_loss', mode='auto', verbose=2, patience = 5)
  batch = 64
  stride = 113
  validation = None
  for i in range(count):
    print(i)
    pre_process = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
    pre_process_valid = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
    X_train = None
    Y_train = None
    cases= files[(i*stride):(i*stride)+stride]
    for j,filename in enumerate(cases):
      totalpath = path + "/" + filename
      temp = np.load(totalpath)
      if X_train is None:
        X_train = temp
        Y_train = np.repeat(labels[(i*stride)+j], temp.shape[0])
      else:
        X_train = np.vstack((X_train, temp))
        Y_train = np.append(Y_train, np.repeat(labels[(i*stride)+j], temp.shape[0]))


    X_train = np.array(X_train).reshape((len(X_train),256,256))

    X_train =np.stack([X_train]*3,axis=3)
    print(X_train.shape)
    print(Y_train.shape)

    X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.1, shuffle=True)

    print(X_train.shape)


    if validation is None:
      validation = X_valid
      validation_labels = Y_valid
    else:
      validation = np.vstack((validation, X_valid))
      validation_labels = np.append(validation_labels, Y_valid)

      
    pre_process_valid.fit(validation)
    pre_process.fit(X_train)

    print(X_train.shape)
    print(Y_train.shape)

  

    history = model.fit(pre_process.flow(X_train,Y_train, batch_size=batch),
                        validation_data=pre_process_valid.flow(validation, validation_labels),
                        epochs = 1, validation_steps = len(validation)/batch,
                        callbacks = [mc, es], steps_per_epoch = len(X_train)/batch)

# Transfer Learning Model

In [0]:
model = VGG16(weights='imagenet',include_top=False)  #pre_trained VGG16 model
input_layer = Input(shape=(256,256,3))
transfer = model(input_layer)

X = Flatten()(transfer)   #Adding Dense trainable layers
X = Dense(1024, activation='relu')(X)
X = Dropout(0.5)(X)
X = Dense(512, activation='relu')(X)
X = Dropout(0.5)(X)
X = Dense(1, activation='sigmoid')(X)

my_model = models.Model(inputs=input_layer, outputs=X)

SGD = optimizers.SGD(learning_rate=0.0001, momentum=0.99) #default learning rate is 0.001
my_model.compile(optimizer= SGD, loss = losses.binary_crossentropy, metrics=['accuracy'])  #should've used F1 score / Recall



58892288/58889256 [==============================] - 0s 0us/step


In [0]:
my_model = models.load_model("/content/gdrive/My Drive/Models/TransferVGG_axial_acl.h5")
transfer_model_train("/content/gdrive/My Drive/Dataset/TrainingSet","axial",my_model,"acl")

5
(4044, 256, 256, 3)
(4044,)
(3639, 256, 256, 3)
(3639, 256, 256, 3)
(3639,)
57/56 [==============================] - 138s 2s/step - loss: 0.4160 - accuracy: 0.8700 - val_loss: 0.5098 - val_accuracy: 0.8304
6
(4054, 256, 256, 3)
(4054,)
(3648, 256, 256, 3)
(3648, 256, 256, 3)
(3648,)
57/57 [==============================] - 125s 2s/step - loss: 0.5794 - accuracy: 0.7834 - val_loss: 0.5314 - val_accuracy: 0.8101
7
(3762, 256, 256, 3)
(3762,)
(3385, 256, 256, 3)
(3385, 256, 256, 3)
(3385,)
53/52 [==============================] - 131s 2s/step - loss: 0.4222 - accuracy: 0.8614 - val_loss: 0.4863 - val_accuracy: 0.8141
8
(3935, 256, 256, 3)
(3935,)
(3541, 256, 256, 3)
(3541, 256, 256, 3)
(3541,)
56/55 [==============================] - 132s 2s/step - loss: 0.5465 - accuracy: 0.7727 - val_loss: 0.4818 - val_accuracy: 0.8313
9
(3730, 256, 256, 3)
(3730,)
(3357, 256, 256, 3)
(3357, 256, 256, 3)
(3357,)
53/52 [==============================] - 130s 2s/step - loss: 0.4470 - accuracy: 0.8439 - 